# Running on a Dask Cluster (with Coiled)

Previously, we expanded each row to the full timeseries to use with the `forecast` function. In practice, we want to be minimizing network data transfer when it comes to distributed computing.

## Creating a Software Environment

```python
import coiled

coiled.create_software_environment(
    name="pydata",
    pip=["fugue[dask]", "statsforecast", "mlflow"],
)
```

## Spinning up a Cluster

In [1]:
from dask.distributed import Client
from coiled import Cluster

# cluster = Cluster(name="pydata", software="pydata", n_workers=2)
# client = Client(cluster)
client = Client()


## Setup

In [2]:
import pandas as pd
import os

download_path = os.path.abspath(os.path.join(".","..","data","m5-forecasting-accuracy.zip"))
unzipped_path = os.path.abspath(os.path.join(".","..","data","m5-forecasting-accuracy-unzipped"))

# Read in the data
INPUT_DIR = unzipped_path
WORKING_DIR = os.path.join(unzipped_path, "..", "working")
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
training_data = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')
sell_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

## Minimizing Data Footprint

In [3]:
from typing import List, Dict, Any, Iterable
from datetime import date

def prices_to_series(df:pd.DataFrame) -> List[Dict[str,Any]]:
    # Assert each date has a price entry
    assert df.shape[0] == (df.date.iloc[-1]-df.date.iloc[0]).days + 1
    return [dict(store_id=df.iloc[0]["store_id"],
                 item_id=df.iloc[0]['item_id'],
                 price_start=df.iloc[0]['date'], 
                 prices=df["sell_price"].tolist())]


df = pd.DataFrame([["store1","item1",date(2020,1,2),2.2], 
                   ["store1","item1",date(2020,1,3),3.3],
                   ["store1","item1", date(2020,1,4),4.4]], 
                   columns=["store_id", "item_id", "date","sell_price"])
print(prices_to_series(df))

[{'store_id': 'store1', 'item_id': 'item1', 'price_start': datetime.date(2020, 1, 2), 'prices': [2.2, 3.3, 4.4]}]


In [4]:
joined = sell_prices.merge(calendar[["date","wm_yr_wk"]], how="inner", on="wm_yr_wk")
joined['date'] = pd.to_datetime(joined['date'])
joined.head()

,store_id,item_id,wm_yr_wk,sell_price,date
0,CA_1,HOBBIES_1_001,11325,9.58,2013-07-13
1,CA_1,HOBBIES_1_001,11325,9.58,2013-07-14
2,CA_1,HOBBIES_1_001,11325,9.58,2013-07-15
3,CA_1,HOBBIES_1_001,11325,9.58,2013-07-16
4,CA_1,HOBBIES_1_001,11325,9.58,2013-07-17


In [6]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
from fugue import transform

out = transform(joined, 
                prices_to_series, 
                schema="store_id:str,item_id:str,price_start:date,prices:[double]",
                partition={"by": ["store_id", "item_id"], "presort": "date asc"})
out.head()

,store_id,item_id,price_start,prices
0,CA_1,FOODS_1_001,2011-01-29,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ..."
1,CA_1,FOODS_1_002,2011-01-29,"[7.88, 7.88, 7.88, 7.88, 7.88, 7.88, 7.88, 7.8..."
2,CA_1,FOODS_1_003,2011-01-29,"[2.88, 2.88, 2.88, 2.88, 2.88, 2.88, 2.88, 2.8..."
3,CA_1,FOODS_1_004,2012-03-03,"[1.78, 1.78, 1.78, 1.78, 1.78, 1.78, 1.78, 1.7..."
4,CA_1,FOODS_1_005,2011-01-29,"[2.94, 2.94, 2.94, 2.94, 2.94, 2.94, 2.94, 2.9..."
